In [1]:
import pydantic
from pydantic import BaseModel
from openff.toolkit import Molecule
from openpharmmdflow.io.load import load_file
from openpharmmdflow.bespokefit import build_bespoke_workflow_factory, run_bespokefit

In [2]:
pydantic.__version__

'1.10.17'

In [3]:
from pathlib import Path


class SmallMoleculePipelineInputConfig(BaseModel):
    # TODO: support SMILES
    name: str
    path: Path


class BespokeWorkflowFactoryConfig(BaseModel):
    initial_force_field: str = "openff-2.2.0.offxml"
    qc_method: str = "gfn2xtb"
    qc_basis: str | None = None
    qc_program: str = "xtb"
    qc_spec_name: str = "xtb"
    qc_spec_description: str = "gfn2xtb"


class BespokeExecutorConfig(BaseModel):
    n_fragmenter_workers: int = 1
    n_optimizer_workers: int = 1
    n_qc_compute_workers: int = 2
    n_bespoke_workers: int = 1
    n_fragment_workers: int = 1


class BespokefitConfig(BaseModel):
    # Right now this works to optimize one small molecule
    # we could hack together something that works but it would
    # be best to run this out of band and just load in the
    # bespoke forcefeild
    bespoke_workflow_factory_config: BespokeWorkflowFactoryConfig
    bespoke_executor_config: BespokeExecutorConfig
    save_bespoke_ff: bool = True


class SmallMoleculePipelinePrepConfig(BaseModel):
    bespokefit_config: BespokefitConfig


class SmallMoleculePipelinePackConfig(BaseModel):
    pass


class SmallMoleculePipelineSimulateConfig(BaseModel):
    pass


class SmallMoleculePipelineAnalyizeConfig(BaseModel):
    pass


class SmallMoleculePipelineConfig(BaseModel):
    work_dir: Path
    inputs: list[SmallMoleculePipelineInputConfig] | SmallMoleculePipelineInputConfig
    prep_config: SmallMoleculePipelinePrepConfig | None
    pack_config: SmallMoleculePipelinePackConfig
    simulate_config: SmallMoleculePipelineSimulateConfig
    analyize_config: SmallMoleculePipelineAnalyizeConfig


SMPInputConfig = SmallMoleculePipelineInputConfig
SMPConfig = SmallMoleculePipelineConfig

In [4]:
input = {"name": "ibuprofen", "path": "ibuprofen.sdf"}

sm_input = SMPInputConfig(**input)

settings = {
    "work_dir": "/home/mmh/Projects/OpenPharmMDFlow/experiments/sm/scratch",
    "inputs": [sm_input, sm_input],
    "prep_config": SmallMoleculePipelinePrepConfig(
        BespokefitConfig(
            bespoke_workflow_factory_config=BespokeWorkflowFactoryConfig(),
            bespoke_executor_config=BespokeExecutorConfig(),
        )
    ),
    "pack_config": SmallMoleculePipelinePackConfig(),
    "simulate_config": SmallMoleculePipelineSimulateConfig(),
    "analyize_config": SmallMoleculePipelineAnalyizeConfig(),
}

sm_config = SMPConfig(**settings)

In [5]:
class SmallMoleculePipeline:
    # TODO: Track the stage
    # TODO: serialize
    # TODO: factory model?
    def __init__(self, config: SmallMoleculePipelineInputConfig):
        self.config = config
        self.inputs = (
            config.inputs if isinstance(config.inputs, list) else [config.inputs]
        )
        self.prep_config = config.prep_config if config.prep_config else None

    def load(self):
        self.loaded_mols = []
        for input in self.inputs:
            mol = load_file(input.path)
            mol.name = input.name
            self.loaded_mols.append(mol)

    def prep(self):
        # run bespokefit here
        pass

    def pack(self):
        # build the box here
        pass

    def simulate(self):
        # run simulation here
        pass

    def analyize(self):
        # run analysis here
        pass

In [6]:
smp = SmallMoleculePipeline(sm_config)

In [7]:
smp.load()

In [8]:
mol = Molecule.from_file(sm_input.path)
mol.name = sm_input.name

In [9]:
mol.name

'ibuprofen'

In [10]:
smp.loaded_mols

[Molecule with name 'ibuprofen' and SMILES '[H][O][C](=[O])[C@@]([H])([c]1[c]([H])[c]([H])[c]([C]([H])([H])[C]([H])([C]([H])([H])[H])[C]([H])([H])[H])[c]([H])[c]1[H])[C]([H])([H])[H]',
 Molecule with name 'ibuprofen' and SMILES '[H][O][C](=[O])[C@@]([H])([c]1[c]([H])[c]([H])[c]([C]([H])([H])[C]([H])([C]([H])([H])[H])[C]([H])([H])[H])[c]([H])[c]1[H])[C]([H])([H])[H]']

In [11]:
print(smp.prep_config)

None


In [ ]:
bespoke_workflow_factory_config = BespokeWorkflowFactoryConfig()
bespokefit_config = BespokefitConfig(
    bespoke_workflow_factory_config=BespokeWorkflowFactoryConfig(),
    bespoke_executor_config=BespokeExecutorConfig(),
)
factory = build_bespoke_workflow_factory(bespoke_workflow_factory_config)
ff = run_bespokefit(bespokefit_config, smp.loaded_mols[0], factory)

Building Fitting Schema: 100%|████████████████████| 1/1 [00:00<00:00,  2.09it/s]


Output()

[✓] fragmentation successful

Output()